In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
class Block:
    def __init__(self, id, content, proposer):
        self.id = id #Unique id for each block
        self.content = content #Some random number as block content
        self.signatures = [] #List of validators who sign the block
        self.numberOfSignatures = 0 #Number of signatures
        self.proposer = proposer #Proposer who proposed the block
        self.exNum = 0 #Number of validators who signed the block, but got excluded

    def isValid(self):
        return True

    def isConfirmed(self , size):
        if len(self.signatures) > (2*(size / 3)):
            return True
        return False

In [ ]:
class Validator:
    def __init__ (self , id , type , stake, loss=0, num=0, strategy = "always", group = 0, target = 0, atNoVote = False, atOmitVote = False):
        self.id = id #Validator unique id
        self.type = type #Type of validator, either correct or Byzantine
        self.stake = stake #Total proft that validator gains
        self.lastBlockProposed = -1 #Id of the last proposed block by this validator
        self.numberOfSignutaresFound = 1 
        self.honesty = 0 #Validator reputation
        self.participation = 1 
        self.proposerCount = 0
        self.committeeCount = 0
        self.messageLoss = loss #The probability of the messeges of the validator to get lost
        self.number = num 
        self.proposedBlocks = [] #List of proposed blocks
        self.strategy = strategy #How the validator excludes others
        self.target = target #Who is the validator attacking
        self.group = group #Processes group
        self.atNoVote = atNoVote #For byzantine processes: whether or not they perform the no vote attack. 
        self.atOmitVote = atOmitVote #For byzantine processes: whether ot not they perform the vote omission attack. 

    def propose(self, blocks):
        r = random.randint(0, 100)
        b = Block(len(blocks) , r, self)
        self.proposedBlocks.append(b)
        return b

    def sign(self , NewBlock , total, doAttack):
        if self.type == "Correct":
            if NewBlock.isValid():
                NewBlock.signatures.append(self)
        if self.type == "Under Attack":
            if NewBlock.isValid():
                NewBlock.signatures.append(self)
        elif self.type == "Collusion":
            if NewBlock.isValid():
              if doAttack:
                if self.id%2 ==0:
                  return
                NewBlock.signatures.append(self)
        elif self.type == "Byzantine":
            if NewBlock.isValid():
                if self.atNoVote:
                  if NewBlock.proposer.group == self.target:
                    return
                NewBlock.signatures.append(self)

    def distributeRewardsCorrectly(self, blocks, size, bc, bv):
        PreBlock = blocks[len(blocks) - 1]
        total = 0
        for v in PreBlock.signatures:
            v.stake = (100 / size) + v.stake
        bonus = ((((((len(PreBlock.signatures) / size) * 100 - 66) / 34) * bv) + bc) / 100) * 100
        self.stake = self.stake + bonus
        PreBlock.numberOfSignatures = len(PreBlock.signatures)

    def distributeRewardCollusionally(self , blocks , size, bc, bv):
        PreBlock = blocks[len(blocks) - 1]
        temp = sorted(PreBlock.signatures, key=lambda x: x.stake, reverse=True)
        count = 0
        deleted = []
        for i in range(0, len(PreBlock.signatures)):
            if (len(PreBlock.signatures)-count-1) <= 2*(math.floor((size/3))):
                break
            removedV = temp[i]
            if removedV.type != "Collusion":
                count += 1
                deleted.append(removedV)

        for i in range(0, len(PreBlock.signatures)):
            if PreBlock.signatures[i] not in deleted:
                PreBlock.signatures[i].stake = ((100 / (len(PreBlock.signatures)-count))) + PreBlock.signatures[i].stake
        bonus = (((((((size-count) / size) * 100 - 66) / 34) * bv) + bc) / 100) * 100
        self.stake = self.stake + bonus
        PreBlock.numberOfSignatures = len(PreBlock.signatures) - count
        PreBlock.exNum = count

    def distributeRewardByzantinly(self , blocks , size, bc, bv):
        PreBlock = blocks[len(blocks) - 1]
        temp = sorted(PreBlock.signatures, key=lambda x: x.stake, reverse=True)
        count = 0
        deleted = []
        for i in range(0, len(PreBlock.signatures)):
            if (len(PreBlock.signatures)-count-1) <= 2*(math.floor((size/3))):
                break
            removedV = temp[i]
            if removedV.group == self.target:
                count += 1
                deleted.append(removedV)

        for i in range(0, len(PreBlock.signatures)):
            if PreBlock.signatures[i] not in deleted:
                PreBlock.signatures[i].stake = ((100 / (len(PreBlock.signatures)-count))) + PreBlock.signatures[i].stake
        bonus = (((((((size-count) / size) * 100 - 66) / 34) * bv) + bc) / 100) * 100
        self.stake = self.stake + bonus
        PreBlock.numberOfSignatures = len(PreBlock.signatures) - count
        if PreBlock.numberOfSignatures == 6:
            print("")
        PreBlock.exNum = count

    def distributeRewardsForPreviousBlock(self , blocks , size, T, bc, bv, rho, round):
        deleted = []
        PreBlock = blocks[len(blocks)-1]
        for i in range(0 , len(PreBlock.signatures)):
          r = random.randint(0, 100)
          if r < self.messageLoss:
            deleted.append(PreBlock.signatures[i])
        if len(PreBlock.signatures) - len(deleted) <= 2*(math.floor((size/3))):
            return -1
        for d in deleted:
            PreBlock.signatures.remove(d)
        r = random.randint(0, 100)
        if r < (rho*100):
            if self.type == "Correct" or self.type == "Under Attack":
                self.distributeRewardsCorrectly(blocks , size, bc, bv)
            elif self.type == "Collusion":
                if self.strategy == "everyOtherT":
                  if ((int)(round/T)) % 2 == 1:
                    self.distributeRewardCollusionally(blocks, size, bc, bv)
                  else:
                     self.distributeRewardsCorrectly(blocks , size, bc, bv)
                else:
                  self.distributeRewardCollusionally(blocks, size, bc, bv)
            elif self.type == "Byzantine":
              if self.atOmitVote:
                self.distributeRewardByzantinly(blocks, size, bc, bv)
              else:
                self.distributeRewardsCorrectly(blocks , size, bc, bv)
        else:
            self.distributeRewardsCorrectly(blocks , size, bc, bv)
        return 1

In [ ]:
class Committee:
    def __init__(self, size, users, T, H, selectionType, atNoVote, atOmitVote, bc = 0, bv = 5, lt="Normal", rho = 1, alpha = 1):
        self.committeeSize = size
        self.validators = users
        self.T = T
        self.H = H
        self.blocks = []
        self.selectionType = selectionType
        self.atNoVote = atNoVote
        self.atOmitVote = atOmitVote
        self.bonusC = bc #Constant part of bonus
        self.bonusV = bv #Variational part of bonus
        self.lossType = lt
        self.omittedBlocks = 0
        self.rho = rho #Probability of performing the attack
        self.alpha = alpha #Alpha

    def chooseProposerReputation(self):
        for v in self.validators:
            v.honesty = 1
            v.proposerCount = 0
        for i in range(len(self.blocks)-2, len(self.blocks)-self.T, -1):
            if i<0:
                break
            th = (((self.blocks[i].numberOfSignatures / self.committeeSize) - 0.66) / 0.34)
            th = th ** self.alpha
            self.blocks[i+1].proposer.proposerCount +=1
            self.blocks[i+1].proposer.honesty = ((((self.blocks[i+1].proposer.honesty) * (self.blocks[i+1].proposer.proposerCount-1)) + th) / self.blocks[i+1].proposer.proposerCount)
        try:
            honesties = [v.honesty*self.H for v in self.validators]
            r = random.choices(self.validators, weights=honesties, k=1)
            self.proposer = r[0]
        except:
            print("Oops!", "occurred.")

    def chooseProposerRandom(self):
        r = random.randint(0, len(self.validators) - 1)
        self.proposer = self.validators[r]

    def round(self, roundNumber):
        if self.selectionType == "Reputation":
            self.chooseProposerReputation()
        elif self.selectionType == "Random":
            self.chooseProposerRandom()
        newBlock = self.proposer.propose(self.blocks)
        total = 0
        for v in self.validators:
            v.sign(newBlock, 0, self.atNoVote)
        if newBlock.isConfirmed(self.committeeSize):
            if len(self.blocks)>0:
                ress = self.proposer.distributeRewardsForPreviousBlock(self.blocks , self.committeeSize, self.T, self.bonusC, self.bonusV, self.rho, roundNumber)
                if ress != 1:
                    if ress == 0:
                        self.omittedBlocks += 1
                    return False
            self.blocks.append(newBlock)
            return True
        else:
            print ("Invalid")
            return False

In [ ]:
n = 100
ms = [1, 10, 20, 30, 33]
totalRounds = 60000

In [ ]:
baselineShare = []
for m in ms:
  users = []
  for i in range(0, n-m):
    users.append(Validator(len(users), "Correct", 0 , 0))
  for i in range(0, m):
    users.append(Validator(len(users), "Collusion", 0 , 0))
  committee = Committee(100, users, 10000 , 1000 , "Random", False, True, 0, 0, "Normal", 1)
  for i in range(0, totalRounds):
    print(str(m) +" " + str(i))
    isProduced = committee.round(i)
  CorrectTotal = 0
  ColludingTotal = 0
  for k in range(0, len(users)):
    if users[k].type == "Collusion":
      ColludingTotal += users[k].stake
    if users[k].type == "Correct":
      CorrectTotal += users[k].stake
  CorrectShare = (CorrectTotal / (ColludingTotal + CorrectTotal )) / (n-m)
  ColludingShare = (ColludingTotal / (ColludingTotal + CorrectTotal )) / m
  baselineShare.append(ColludingShare)

In [ ]:
totalRounds = 60000
baselineShare = []
users = []
for i in range(0, 65):
  users.append(Validator(len(users), "Correct", 0 , 0, 0, "always", 1, 0, False, False))
for i in range(0, 1):
  users.append(Validator(len(users), "Correct", 0 , 0, 0, "always", 2, 0, False, False))
for i in range(0, 5):
  users.append(Validator(len(users), "Correct", 0 , 0, 0, "always", 3, 0, False, False))
for i in range(0, 17):
  users.append(Validator(len(users), "Correct", 0 , 0, 0, "always", 4, 0, False, False))
for i in range(0, 10):
  users.append(Validator(len(users), "Byzantine", 0 , 0, 0, "always", 5, 2, False, True))
for i in range(0, 20):
  users.append(Validator(len(users), "Byzantine", 0 , 0, 0, "always", 6, 3, False, True))
for i in range(0, 10):
  users.append(Validator(len(users), "Byzantine", 0 , 0, 0, "always", 7, 4, False, True))
committee = Committee(128, users, 10000 , 1000 , "Reputation", False, True, 0, 0, "Normal", 1, 15)
for i in range(0, totalRounds):
  print(str(i))
  isProduced = committee.round(i)
print(committee.omittedBlocks)

total= [0,0,0,0,0,0,0]
nums=[65,1,5,17,10,20,10]
for k in range(1, 8):
  for j in range(0 , len(users)):
    if users[j].group == k:
      total[k-1] += users[j].stake
print(len(users))
repuShare3 = [total[j]/ sum(total) for j in range(len(total))]
repuShare3 = [repuShare3[j]/ nums[j] for j in range(len(repuShare3))]